## pip install 
#### xmltodict
#### networkx
#### tqdm

In [1]:
import os
import pandas as pd
import sys
import regex as re
import numpy as np
import datetime
import time
from ffiec_data_connect import methods, credentials, ffiec_connection
creds = credentials.WebserviceCredentials(username="whills12", password="vGzHZX7EAsPCwfXuV6pI")
conn = ffiec_connection.FFIECConnection()

In [2]:
import snowflake.connector as snow

In [5]:
def extract_taxonomy_data(df, form_id):
    data_list = []
    for i in range(len(df)):
        mdrm_code = re.split('_',df.index[i])[1]
        keys_list = list(df["data"][i].keys())
        schedule_type = keys_list[0]
        subset_of_data = df["data"][i][keys_list[0]]
        schedule_label = subset_of_data['line_ids']['schedule']['label']
        mdrm_label = subset_of_data['line_ids']['extra_col_0']['label']
        if len(subset_of_data['line_ids']) > 2: 
            j = 1 
            while True:
                try:
                    mdrm_label += subset_of_data['line_ids'][f'extra_col_{j}']['label']
                    j += 1
                except TypeError:
                    j+=1
                    continue
                except KeyError:
                    break
        data_list.append({
            'mdrm_code': mdrm_code,
            'schedule_type': schedule_type,
            'schedule_label': schedule_label,
            'mdrm_label': mdrm_label,
            'form_type': form_id
        })
    final_df = pd.DataFrame(data_list)
    return final_df

In [6]:
def match_data_type(id_rssd_set, mdrm_set):
    master_data_type_list = []
    trials = 0
    print("The time of code execution begin is : ", time.ctime())
    while len(mdrm_set) != 0 and trials < len(id_rssd_set):
        for id_rssd in id_rssd_set:
                time_series = methods.collect_data(
                session=conn,
                creds=creds,
                rssd_id=id_rssd,
                reporting_period="9/30/2023",
                series="call",
                output_type='pandas'
                )
                for mdrm in mdrm_set.copy():
                    if mdrm in time_series['mdrm'].values:
                        data_type = time_series['data_type'][time_series['mdrm']==mdrm].values[0]
                        pair_mdrm_type = (mdrm,data_type)
                        master_data_type_list.append(pair_mdrm_type)
                        mdrm_set.remove(mdrm)
                trials +=1
                if not mdrm_set:
                    break
                time.sleep(1.5)
    print("The time of code execution end was : ", time.ctime())
    return master_data_type_list

In [1]:
def filter_mdrm(path):
    mdrm_csv = pd.read_csv(path, skiprows=1)
    mdrm_csv.insert(2, 'mdrm_code', mdrm_csv['Mnemonic'] + mdrm_csv['Item Code'])
    mdrm_filtered = mdrm_csv[(mdrm_csv['Reporting Form'].isin(in_report)) & (~mdrm_csv['Mnemonic'].isin(not_mnemonic))]
    mdrm_filtered.rename(columns={"Reporting Form": "form_type"},inplace=True)
    return mdrm_filtered

In [5]:
def create_table_query(table_name, columns_dict):
    columns = list(columns_dict.keys())
    data_types = list(columns_dict.values())

    if len(columns) != len(data_types):
        raise ValueError("Number of columns and data types must be the same.")

    columns_str = ', '.join([f'{col} {data_type}' for col, data_type in zip(columns, data_types)])
    query = f'CREATE TABLE IF NOT EXISTS {table_name} ({columns_str}, primary key (IDRSSD, RCON9999));'

    return query

In [6]:
def gather_all_queries(final_df):
    create_queries_list= []
    schedules = set(final_df['schedule_type'])
    for s in schedules:
        schedule_dict_list = {sd['mdrm_code']: sd['data_type'] for sd in final_df.loc[final_df['schedule_type'].isin([s]), ['mdrm_code', 'data_type']].to_dict(orient='records')}
        bank_id = 'IDRSSD'
        bank_id_data_type = 'VARCHAR'
        report_date = 'RCON9999'
        report_date_data_type= 'DATE'
        for key,item in schedule_dict_list.items():
            if item == 'str':
                schedule_dict_list[key] = 'VARCHAR'
            elif item in ['float','int']:
                schedule_dict_list[key] = 'FLOAT'
            elif item == 'bool':
                schedule_dict_list[key] = 'BOOLEAN'   
        if bank_id in schedule_dict_list:
            my_dict[bank_id] = bank_id_data_type
        else:
            schedule_dict_list[bank_id] = bank_id_data_type
        if report_date in schedule_dict_list:
            schedule_dict_list[report_date] = report_date_data_type
        else:
            schedule_dict_list[report_date] = report_date_data_type
        create_queries_list.append(create_table_query(f'Schedule_{s}', schedule_dict_list))
    return create_queries_list

In [7]:
full_query_list = gather_all_queries(final_df)

In [8]:
for query in full_query_list:
    print(query)

CREATE TABLE IF NOT EXISTS Schedule_RIE (RIAD0497 FLOAT, RIAD4042 FLOAT, RIAD4136 FLOAT, RIAD4141 FLOAT, RIAD4146 VARCHAR, RIAD4461 FLOAT, RIAD4462 FLOAT, RIAD4463 FLOAT, RIAD4464 FLOAT, RIAD4467 FLOAT, RIAD4468 FLOAT, RIAD4486 VARCHAR, RIAD4487 VARCHAR, RIAD4488 VARCHAR, RIAD4489 VARCHAR, RIAD4491 VARCHAR, RIAD4498 FLOAT, RIAD4499 FLOAT, RIAD4521 FLOAT, RIAD4522 FLOAT, RIAD4769 BOOLEAN, RIAD8403 FLOAT, RIADB526 FLOAT, RIADB527 FLOAT, RIADC013 FLOAT, RIADC014 FLOAT, RIADC015 FLOAT, RIADC016 FLOAT, RIADC017 FLOAT, RIADC018 FLOAT, RIADF555 FLOAT, RIADF556 FLOAT, RIADF557 FLOAT, RIADF558 FLOAT, RIADF559 FLOAT, RIADFT29 FLOAT, RIADFT30 FLOAT, RIADFT31 FLOAT, RIADFT32 FLOAT, RIADJJ26 FLOAT, RIADJJ27 FLOAT, RIADJJ28 FLOAT, RIADT047 FLOAT, RIADY923 FLOAT, RIADY924 FLOAT, IDRSSD VARCHAR, RCON9999 DATE, primary key (IDRSSD, RCON9999));
CREATE TABLE IF NOT EXISTS Schedule_RCRI (RCFA2170 FLOAT, RCFA3128 FLOAT, RCFA3792 FLOAT, RCFA5310 FLOAT, RCFA5311 FLOAT, RCFA7204 FLOAT, RCFA7205 FLOAT, RCFA720

In [11]:
conn= snow.connect(user="WALKERHILLS", password="h!123@iLLs#", account="lrlwqdb-yib23397")

In [9]:
def main(conn, full_query_list):
    cur = conn.cursor()
    sql = "USE ROLE ACCOUNTADMIN"
    cur.execute(sql)
    sql = "USE WAREHOUSE COMPUTE_WH"
    cur.execute(sql)
    sql = "ALTER WAREHOUSE COMPUTE_WH RESUME IF SUSPENDED"
    cur.execute(sql)
    sql = "CREATE DATABASE IF NOT EXISTS CALLREPORTDB"
    cur.execute(sql)
    sql = "USE DATABASE CALLREPORTDB"
    cur.execute(sql)
    sql = "CREATE SCHEMA IF NOT EXISTS PUBLIC"
    cur.execute(sql)
    sql = "USE SCHEMA PUBLIC"
    cur.execute(sql)
    for query in full_query_list:
        cur.execute(query)
    sql = "ALTER WAREHOUSE COMPUTE_WH SUSPEND"
    cur.execute(sql)
    cur.close()
    conn.close()

In [12]:
if __name__ == "__main__":
    main(conn,full_query_list)

In [ ]:
def main(session: snowpark.Session): 
    # CREATE TABLE
    create_table_query  = f"""
       CREATE OR REPLACE TABLE TEST_DB.TEST_SCHEMA.CALL_CENTER_COPY AS
       SELECT 
              CC_CALL_CENTER_ID
             ,CC_REC_START_DATE 
             ,CC_REC_END_DATE
       FROM TEST_DB.TEST_SCHEMA.CALL_CENTER;
    """
    session.sql(create_table_query).collect()
    # SELECT TABLE
    create_table_query  = f"""
       SELECT 
              *
       FROM TEST_DB.TEST_SCHEMA.CALL_CENTER_COPY;
    """
    call_center_table = session.sql(create_table_query).collect()
    return call_center_table

In [ ]:
mdrm_filtered = filter_mdrm(path="C:\\Users\\whill\\call-report-database\\MDRM_CSV.csv")

In [5]:
filers = methods.collect_filers_on_reporting_period(
    session=conn,
    creds=creds,
    reporting_period="9/30/2023",
    output_type='pandas'
)
filers

id_rssd fdic_cert_number occ_chart_number ots_dock_number  \
0          37            10057             None           16553   
1         242             3850             None            None   
2         279            28868             None            2523   
3         354            14083             None            None   
4         457            10202             None            None   
...       ...              ...              ...             ...   
4664  5805479            59346            25287            None   
4665  5805488            59349            25275            None   
4666  5805817            59337            25237            None   
4667  5859511            59344            25288            None   
4668  5860740            59305             None            None   

     primary_aba_rout_number  \
0                   61107146   
1                   81220537   
2                  311972526   
3                  101107475   
4                   91208332   
...                      ...   
4664               122402463   
4665               121202457   
4666                91018470   
4667                53012977   
4668                72414514   

                                                   name state         city  \
0                                BANK OF HANCOCK COUNTY    GA       SPARTA   
1                      FIRST COMMUNITY BANK XENIA-FLORA    IL        XENIA   
2                           MINEOLA COMMUNITY BANK, SSB    TX      MINEOLA   
3                                      BISON STATE BANK    KS        BISON   
4                                      LOWRY STATE BANK    MN        LOWRY   
...                                                 ...   ...          ...   
4664  BESSEMER TRUST COMPANY OF NEVADA, NATIONAL ASS...    NV    LAS VEGAS   
4665        INSPIRE TRUST COMPANY, NATIONAL ASSOCIATION    NV         RENO   
4666                       CERIDIAN NATIONAL TRUST BANK    MN  BLOOMINGTON   
4667                   TIAA TRUST, NATIONAL ASSOCIATION    NC    CHARLOTTE   
4668                               COMMUNITY UNITY BANK    MI   BIRMINGHAM   

                                                address filing_type  \
0                                    12855 BROAD STREET         051   
1                                      260 FRONT STREET         051   
2                                           215 W BROAD         051   
3                                       223 MAIN STREET         051   
4                                     400 FLORENCE AVE.         051   
...                                                 ...         ...   
4664  1700 SOUTH PAVILION CENTER DRIVE         SUITE...         051   
4665                         241 RIDGE STREET SUITE 310         041   
4666                        3311 EAST OLD SHAKOPEE ROAD         051   
4667                     8500 ANDREW CARNEGIE BOULEVARD         051   
4668                              34040 WOODWARD AVENUE         051   

      has_filed_for_reporting_period  
0                               True  
1                               True  
2                               True  
3                               True  
4                               True  
...                              ...  
4664                            True  
4665                            True  
4666                            True  
4667                            True  
4668                            True  

[4669 rows x 11 columns]

In [44]:
time_series = methods.collect_data(
                session=conn,
                creds=creds,
                rssd_id=37,
                reporting_period="9/30/2023",
                series="call",
                output_type='pandas'
                )
time_series[time_series['mdrm']=='RSSD9017']

mdrm rssd    quarter  int_data  float_data bool_data  \
53  RSSD9017   37  9/30/2023       NaN         NaN      None   

                  str_data data_type  
53  BANK OF HANCOCK COUNTY       str

In [6]:
id_rssd_set = set(filers['id_rssd'])

In [30]:
len(id_rssd_set)

4669

In [35]:
for id_rssd in id_rssd_set:
    print(str(id_rssd))

2986407
987231
2806840
143952
835378
701857
753052
5805488
506922
686459
687830
3952904
358112
568135
37341
2921211
527954
959630
21658
175609
397559
754246
785950
109453
823478
764151
2533119
2193616
147679
498148
717232
757452
399357
786555
920733
692946
680055
152253
482156
624778
1188772
189129
233358
106452
141677
547251
88950
956956
126012
463650
228000
814074
814711
3153233
719030
960560
760939
568377
733054
3268249
176464
3214059
573036
166849
280558
479268
179335
90047
118156
910079
505
965789
22048
333650
584377
837653
1879016
2719810
44741
37051
282226
459671
724744
775773
43052
904845
550635
999935
207470
655839
578116
3216316
5069
83834
354057
470050
739832
2916534
384018
33370
311603
1391778
331076
776369
627854
128258
325141
738143
208459
916745
3476192
820879
235174
513340
33259
336147
145554
720157
768476
2819242
732954
98351
68671
875132
581237
432058
924553
919568
921178
252452
715470
771140
137205
112332
780049
319047
411576
135836
5205819
64150
744854
1006513
12647

In [ ]:
result_df = pd.DataFrame(columns=df['mdrm'].unique())

# Iterate through each row in the original DataFrame
for index, row in df.iterrows():
    mdrm = row['mdrm']
    data_type = row['data_type']

    # Depending on the 'data_type', select the corresponding value
    if data_type == 'str':
        value = row['str_data']
    elif data_type == 'bool':
        value = row['bool_data']
    elif data_type == 'float':
        value = row['float_data']
    elif data_type == 'int':
        value = row['int_data']
    else:
        value = None

    # Set the value in the result DataFrame
    result_df.at[0, mdrm] = value

# Transpose the result DataFrame to have one row and 583 columns
#result_df = result_df.T

# Reset the index and rename columns
#result_df = result_df.reset_index()
#result_df.columns = ['mdrm', 0]

# Display the resulting DataFrame
print(result_df)

In [17]:
datetime.datetime(9999,12,31)

datetime.datetime(9999, 12, 31, 0, 0)

In [8]:
not_mnemonic= ['TEXT', 'TE01', 'TE02', 'TE03','TE04','TE05','TE06','TE07','TE08','TE09','TE10','TE11','TE12']
in_report = ['FFIEC 031', 'FFIEC041','FFIEC 051']

In [9]:
mdrm_csv = pd.read_csv("C:\\Users\\whill\\call-report-database\\MDRM_CSV.csv", skiprows=1)
mdrm_csv.insert(2, 'mdrm_code', mdrm_csv['Mnemonic'] + mdrm_csv['Item Code'])
mdrm_csv

Mnemonic Item Code mdrm_code      Start Date         End Date  \
0         AAAA      2170  AAAA2170  9/30/2016 0:00  12/31/9999 0:00   
1         AAAA      5310  AAAA5310  3/31/2014 0:00  12/31/9999 0:00   
2         AAAA      7205  AAAA7205  3/31/2014 0:00  12/31/9999 0:00   
3         AAAA      7206  AAAA7206  3/31/2014 0:00  12/31/9999 0:00   
4         AAAA      8274  AAAA8274  9/30/2016 0:00  12/31/9999 0:00   
...        ...       ...       ...             ...              ...   
88145     XMTA      9191  XMTA9191   2/1/2010 0:00  12/31/9999 0:00   
88146     XMTA      9200  XMTA9200   5/4/2015 0:00  12/31/9999 0:00   
88147     XMTA      9220  XMTA9220   5/4/2015 0:00  12/31/9999 0:00   
88148     XMTA      9912  XMTA9912   5/4/2015 0:00  12/31/9999 0:00   
88149     XMTA      9999  XMTA9999   5/4/2015 0:00  12/31/9999 0:00   

                                               Item Name Confidentiality  \
0                                           TOTAL ASSETS               N   
1            GENERAL LOAN AND LEASE VALUATION ALLOWANCES               N   
2                         TOTAL RISK-BASED CAPITAL RATIO               N   
3                        TIER 1 RISK-BASED CAPITAL RATIO               N   
4      TIER 1 CAPITAL ALLOWABLE UNDER THE RISK-BASED ...               N   
...                                                  ...             ...   
88145                                PROCESSING DISTRICT               Y   
88146                             ABBREVIATED STATE NAME               Y   
88147                  PHYSICAL ZIP/FOREIGN MAILING CODE               Y   
88148                                       DATE OF LOAN               Y   
88149                       REPORTING DATE (CC;YR;MO;DA)               Y   

      ItemType Reporting Form  \
0            F      FFIEC 101   
1            F      FFIEC 101   
2            P      FFIEC 101   
3            P      FFIEC 101   
4            F      FFIEC 101   
...        ...            ...   
88145        S     FR 2028a/s   
88146        F     FR 2028a/s   
88147        F     FR 2028a/s   
88148        F     FR 2028a/s   
88149        F     FR 2028a/s   

                                             Description  \
0      The sum of all asset items. This item must equ...   
1       Includes general loan and lease valuation all...   
2      This ratio includes total risk-based capital d...   
3      This ratio includes Tier 1 (core) capital; les...   
4       Includes the amount of Tier 1 capital less de...   
...                                                  ...   
88145  Effective 10/24/2011; the FR 2572 information ...   
88146  Two character state abbreviation as assigned t...   
88147  The zip code or foreign mailing code of the ph...   
88148  Includes the calendar date the funds reported ...   
88149  Effective 10/24/2011; the FR 2572 information ...   

                                          SeriesGlossary  
0                 Advanced Approaches Regulatory Capital  
1                 Advanced Approaches Regulatory Capital  
2                 Advanced Approaches Regulatory Capital  
3                 Advanced Approaches Regulatory Capital  
4                 Advanced Approaches Regulatory Capital  
...                                                  ...  
88145   NOTE: \r\n\r\nThe mnemonic prefixes in this f...  
88146   NOTE: \r\n\r\nThe mnemonic prefixes in this f...  
88147   NOTE: \r\n\r\nThe mnemonic prefixes in this f...  
88148   NOTE: \r\n\r\nThe mnemonic prefixes in this f...  
88149   NOTE: \r\n\r\nThe mnemonic prefixes in this f...  

[88150 rows x 11 columns]

In [10]:
mdrm_filtered = mdrm_csv[(mdrm_csv['Reporting Form'].isin(in_report)) & (~mdrm_csv['Mnemonic'].isin(not_mnemonic))]
mdrm_filtered.rename(columns={"Reporting Form": "form_type"},inplace=True)
mdrm_filtered

C:\Users\whill\AppData\Local\Temp\ipykernel_3188\2873219326.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mdrm_filtered.rename(columns={"Reporting Form": "form_type"},inplace=True)


Mnemonic Item Code mdrm_code       Start Date         End Date  \
10710     CALL      8002  CALL8002   3/31/1984 0:00   3/31/2006 0:00   
10714     CALL      8786  CALL8786   3/31/1984 0:00  12/31/9999 0:00   
10716     CALL      8787  CALL8787  12/31/1959 0:00  12/31/9999 0:00   
10717     CALL      8798  CALL8798  12/31/1959 0:00  12/31/9999 0:00   
10718     CALL      8799  CALL8799  12/31/1959 0:00  12/31/2000 0:00   
...        ...       ...       ...              ...              ...   
69998     RSSD      9130  RSSD9130   3/31/2017 0:00  12/31/9999 0:00   
70063     RSSD      9200  RSSD9200   9/30/2005 0:00  12/31/9999 0:00   
70065     RSSD      9200  RSSD9200   3/31/2017 0:00  12/31/9999 0:00   
70084     RSSD      9220  RSSD9220   9/30/2005 0:00  12/31/9999 0:00   
70086     RSSD      9220  RSSD9220   3/31/2017 0:00  12/31/9999 0:00   

                               Item Name Confidentiality ItemType  form_type  \
10710          REPORTING STATUS (W-CODE)               N        F  FFIEC 031   
10714               REPORTING LEVEL CODE               N        S  FFIEC 031   
10716           REPORTING EXCEPTION CODE               N        F  FFIEC 031   
10717                     FDR TIME STAMP               Y        F  FFIEC 031   
10718                ARCHIVAL DATE STAMP               Y        F  FFIEC 031   
...                                  ...             ...      ...        ...   
69998                CITY/TOWN TEXT NAME               N        S  FFIEC 051   
70063             ABBREVIATED STATE NAME               N        S  FFIEC 031   
70065             ABBREVIATED STATE NAME               N        S  FFIEC 051   
70084  PHYSICAL ZIP/FOREIGN MAILING CODE               N        S  FFIEC 031   
70086  PHYSICAL ZIP/FOREIGN MAILING CODE               N        S  FFIEC 051   

                                             Description  \
10710  This item is an asset threshold used to determ...   
10714  Effective 10/24/2011; the FR 2572 information ...   
10716  Effective 10/24/2011; the FR 2572 information ...   
10717  Effective 10/24/2011; the FR 2572 information ...   
10718  The date (ccyymmdd) the data were last updated...   
...                                                  ...   
69998  The name of the city or town in which the enti...   
70063  Two character state abbreviation as assigned t...   
70065  Two character state abbreviation as assigned t...   
70084  The zip code or foreign mailing code of the ph...   
70086  The zip code or foreign mailing code of the ph...   

                                          SeriesGlossary  
10710  \r\nThe following items may be accessed by the...  
10714  \r\nThe following items may be accessed by the...  
10716  \r\nThe following items may be accessed by the...  
10717  \r\nThe following items may be accessed by the...  
10718  \r\nThe following items may be accessed by the...  
...                                                  ...  
69998                                                     
70063                                                     
70065                                                     
70084                                                     
70086                                                     

[6062 rows x 11 columns]

In [ ]:
df1 = extract_taxonomy_data(pd.read_json("C:\\Users\\whill\\call-report-database\\python\\cdr_taxonomy_xbrl_to_json\\example\\031_2023-09-30.json"),'FFIEC 031')
df2 = extract_taxonomy_data(pd.read_json("C:\\Users\\whill\\call-report-database\\python\\cdr_taxonomy_xbrl_to_json\\example\\041_2023-09-30.json"),'FFIEC 041')
df3 = extract_taxonomy_data(pd.read_json("C:\\Users\\whill\\call-report-database\\python\\cdr_taxonomy_xbrl_to_json\\example\\051_2023-09-30.json"),'FFIEC 051')
merged_df = pd.concat([df1,df2,df3]).reset_index(drop=True)
df1 = extract_taxonomy_data(pd.read_json("C:\\Users\\whill\\call-report-database\\python\\cdr_taxonomy_xbrl_to_json\\example\\031_2001-03-31.json"),'FFIEC 031')
df2 = extract_taxonomy_data(pd.read_json("C:\\Users\\whill\\call-report-database\\python\\cdr_taxonomy_xbrl_to_json\\example\\041_2001-03-31.json"),'FFIEC 041')
older_df = pd.concat([df1,df2]).reset_index(drop=True)
df_2000_2023 = pd.concat([merged_df,older_df]).reset_index(drop=True)
df_2000_2023.drop_duplicates(subset=['mdrm_code', 'form_type'],keep='first',inplace=True,ignore_index=True)
rows_to_drop = np.where(df_2000_2023['mdrm_code'].str.contains('TE', regex=True))
df_2000_2023.drop(index=rows_to_drop[0],inplace =True)
result_df = mdrm_filtered.join(df_2000_2023.set_index(['mdrm_code', 'form_type']), on=['mdrm_code','form_type'],how='left')
#result_df[~result_df['schedule_type'].isnull()]
master_data_type_list = match_data_type(id_rssd_set,set(result_df['mdrm_code']))
datatype_df = pd.DataFrame(master_data_type_list,columns=['mdrm_code', 'data_type'])
final_df = result_df.join(datatype_df.set_index(['mdrm_code']), on=['mdrm_code'],how='left')
final_df

# code merged together and written above

In [3]:
final_df = pd.read_csv("C:\\Users\\whill\\call-report-database\\MDRM_CSV_Filtered_Schedules_Data_Types.csv")
final_df

Mnemonic Item Code mdrm_code       Start Date         End Date  \
0        CALL      8002  CALL8002   3/31/1984 0:00   3/31/2006 0:00   
1        CALL      8786  CALL8786   3/31/1984 0:00  12/31/9999 0:00   
2        CALL      8787  CALL8787  12/31/1959 0:00  12/31/9999 0:00   
3        CALL      8798  CALL8798  12/31/1959 0:00  12/31/9999 0:00   
4        CALL      8799  CALL8799  12/31/1959 0:00  12/31/2000 0:00   
...       ...       ...       ...              ...              ...   
6057     RSSD      9130  RSSD9130   3/31/2017 0:00  12/31/9999 0:00   
6058     RSSD      9200  RSSD9200   9/30/2005 0:00  12/31/9999 0:00   
6059     RSSD      9200  RSSD9200   3/31/2017 0:00  12/31/9999 0:00   
6060     RSSD      9220  RSSD9220   9/30/2005 0:00  12/31/9999 0:00   
6061     RSSD      9220  RSSD9220   3/31/2017 0:00  12/31/9999 0:00   

                              Item Name Confidentiality ItemType  form_type  \
0             REPORTING STATUS (W-CODE)               N        F  FFIEC 031   
1                  REPORTING LEVEL CODE               N        S  FFIEC 031   
2              REPORTING EXCEPTION CODE               N        F  FFIEC 031   
3                        FDR TIME STAMP               Y        F  FFIEC 031   
4                   ARCHIVAL DATE STAMP               Y        F  FFIEC 031   
...                                 ...             ...      ...        ...   
6057                CITY/TOWN TEXT NAME               N        S  FFIEC 051   
6058             ABBREVIATED STATE NAME               N        S  FFIEC 031   
6059             ABBREVIATED STATE NAME               N        S  FFIEC 051   
6060  PHYSICAL ZIP/FOREIGN MAILING CODE               N        S  FFIEC 031   
6061  PHYSICAL ZIP/FOREIGN MAILING CODE               N        S  FFIEC 051   

                                            Description  \
0     This item is an asset threshold used to determ...   
1     Effective 10/24/2011; the FR 2572 information ...   
2     Effective 10/24/2011; the FR 2572 information ...   
3     Effective 10/24/2011; the FR 2572 information ...   
4     The date (ccyymmdd) the data were last updated...   
...                                                 ...   
6057  The name of the city or town in which the enti...   
6058  Two character state abbreviation as assigned t...   
6059  Two character state abbreviation as assigned t...   
6060  The zip code or foreign mailing code of the ph...   
6061  The zip code or foreign mailing code of the ph...   

                                         SeriesGlossary schedule_type  \
0     \r\nThe following items may be accessed by the...           NaN   
1     \r\nThe following items may be accessed by the...           NaN   
2     \r\nThe following items may be accessed by the...           NaN   
3     \r\nThe following items may be accessed by the...           NaN   
4     \r\nThe following items may be accessed by the...           NaN   
...                                                 ...           ...   
6057                                                              ENT   
6058                                                              ENT   
6059                                                              ENT   
6060                                                              ENT   
6061                                                              ENT   

                                     schedule_label          mdrm_label  \
0                                               NaN                 NaN   
1                                               NaN                 NaN   
2                                               NaN                 NaN   
3                                               NaN                 NaN   
4                                               NaN                 NaN   
...                                             ...                 ...   
6057  Bank Demographic Information(Form Type - 051)                City   
6058  Bank Demogr

## lets define the last of our data types based on the item types provided for us R- rate stored as a decimal so obviously we need float, for easy we are going to define D and R as float, I went and checked the 443 remaining blank for D and checked that they would be mostly numerical. Then we will set everything to str for remain F and S codes, finally we will set the last schedule as undefined!

In [4]:
final_df.loc[(final_df['data_type'].isnull()) & ((final_df['ItemType']=='D') | (final_df['ItemType']=='R')), 'data_type']='float'
final_df.loc[(final_df['data_type'].isnull()) & ((final_df['ItemType']=='F') | (final_df['ItemType']=='S')), 'data_type']='str'
final_df.loc[final_df['schedule_type'].isnull(),'schedule_type']='Undefined'

In [40]:
schedule_dict_list = {sd['mdrm_code']: sd['data_type'] for sd in final_df.loc[final_df['schedule_type'].isin(['ENT']), ['mdrm_code', 'data_type']].to_dict(orient='records')}
bank_id = 'IDRSSD'
bank_id_data_type = 'VARCHAR'
report_date = 'RCON9999'
report_date_data_type= 'DATE'
for key,item in schedule_dict_list.items():
    if item == 'str':
        schedule_dict_list[key] = 'VARCHAR'
    elif item in ['float','int']:
        schedule_dict_list[key] = 'NUMBER'
    elif item == 'bool':
        schedule_dict_list[key] = 'BOOLEAN'   
if bank_id in schedule_dict_list:
    my_dict[bank_id] = bank_id_data_type
else:
    schedule_dict_list[bank_id] = bank_id_data_type
if report_date in schedule_dict_list:
    schedule_dict_list[report_date] = report_date_data_type
else:
    schedule_dict_list[report_date] = report_date_data_type
schedule_dict_list

{'RCON9224': 'VARCHAR',
 'RCON9999': 'DATE',
 'RSSD9017': 'VARCHAR',
 'RSSD9130': 'VARCHAR',
 'RSSD9200': 'VARCHAR',
 'RSSD9220': 'VARCHAR',
 'IDRSSD': 'VARCHAR'}

In [44]:
create_table_query('Schedule ENT', schedule_dict_list)

'CREATE TABLE Schedule ENT (RCON9224 VARCHAR, RCON9999 DATE, RSSD9017 VARCHAR, RSSD9130 VARCHAR, RSSD9200 VARCHAR, RSSD9220 VARCHAR, IDRSSD VARCHAR), primary key (IDRSSD, RCON9999));'

In [14]:
start_list = [('IDRSSD','VARCHAR'), ('RCON9999','DATE')]
start_list.extend(final_df['mdrm_code'][final_df['schedule_type'].isin(['ENT'])].to_list())
columns_set = set(start_list)
columns_set

Mnemonic Item Code mdrm_code       Start Date         End Date  \
0        CALL      8002  CALL8002   3/31/1984 0:00   3/31/2006 0:00   
1        CALL      8786  CALL8786   3/31/1984 0:00  12/31/9999 0:00   
2        CALL      8787  CALL8787  12/31/1959 0:00  12/31/9999 0:00   
3        CALL      8798  CALL8798  12/31/1959 0:00  12/31/9999 0:00   
4        CALL      8799  CALL8799  12/31/1959 0:00  12/31/2000 0:00   
...       ...       ...       ...              ...              ...   
6057     RSSD      9130  RSSD9130   3/31/2017 0:00  12/31/9999 0:00   
6058     RSSD      9200  RSSD9200   9/30/2005 0:00  12/31/9999 0:00   
6059     RSSD      9200  RSSD9200   3/31/2017 0:00  12/31/9999 0:00   
6060     RSSD      9220  RSSD9220   9/30/2005 0:00  12/31/9999 0:00   
6061     RSSD      9220  RSSD9220   3/31/2017 0:00  12/31/9999 0:00   

                              Item Name Confidentiality ItemType  form_type  \
0             REPORTING STATUS (W-CODE)               N        F  FFIEC 031   
1                  REPORTING LEVEL CODE               N        S  FFIEC 031   
2              REPORTING EXCEPTION CODE               N        F  FFIEC 031   
3                        FDR TIME STAMP               Y        F  FFIEC 031   
4                   ARCHIVAL DATE STAMP               Y        F  FFIEC 031   
...                                 ...             ...      ...        ...   
6057                CITY/TOWN TEXT NAME               N        S  FFIEC 051   
6058             ABBREVIATED STATE NAME               N        S  FFIEC 031   
6059             ABBREVIATED STATE NAME               N        S  FFIEC 051   
6060  PHYSICAL ZIP/FOREIGN MAILING CODE               N        S  FFIEC 031   
6061  PHYSICAL ZIP/FOREIGN MAILING CODE               N        S  FFIEC 051   

                                            Description  \
0     This item is an asset threshold used to determ...   
1     Effective 10/24/2011; the FR 2572 information ...   
2     Effective 10/24/2011; the FR 2572 information ...   
3     Effective 10/24/2011; the FR 2572 information ...   
4     The date (ccyymmdd) the data were last updated...   
...                                                 ...   
6057  The name of the city or town in which the enti...   
6058  Two character state abbreviation as assigned t...   
6059  Two character state abbreviation as assigned t...   
6060  The zip code or foreign mailing code of the ph...   
6061  The zip code or foreign mailing code of the ph...   

                                         SeriesGlossary schedule_type  \
0     \r\nThe following items may be accessed by the...     Undefined   
1     \r\nThe following items may be accessed by the...     Undefined   
2     \r\nThe following items may be accessed by the...     Undefined   
3     \r\nThe following items may be accessed by the...     Undefined   
4     \r\nThe following items may be accessed by the...     Undefined   
...                                                 ...           ...   
6057                                                              ENT   
6058                                                              ENT   
6059                                                              ENT   
6060                                                              ENT   
6061                                                              ENT   

                                     schedule_label          mdrm_label  \
0                                               NaN                 NaN   
1                                               NaN                 NaN   
2                                               NaN                 NaN   
3                                               NaN                 NaN   
4                                               NaN                 NaN   
...                                             ...                 ...   
6057  Bank Demographic Information(Form Type - 051)                City   
6058  Bank Demogr

# break

In [11]:
df1 = extract_taxonomy_data(pd.read_json("C:\\Users\\whill\\call-report-database\\python\\cdr_taxonomy_xbrl_to_json\\example\\031_2023-09-30.json"),'FFIEC 031')
df2 = extract_taxonomy_data(pd.read_json("C:\\Users\\whill\\call-report-database\\python\\cdr_taxonomy_xbrl_to_json\\example\\041_2023-09-30.json"),'FFIEC 041')
df3 = extract_taxonomy_data(pd.read_json("C:\\Users\\whill\\call-report-database\\python\\cdr_taxonomy_xbrl_to_json\\example\\051_2023-09-30.json"),'FFIEC 051')
merged_df = pd.concat([df1,df2,df3]).reset_index(drop=True)
merged_df

mdrm_code schedule_type  \
0     RCFA2170          RCRI   
1     RCFA3128          RCRI   
2     RCFA3792          RCRI   
3     RCFA5310          RCRI   
4     RCFA5311          RCRI   
...        ...           ...   
5933  TEXTFT31           RIE   
5934  TEXTFT42            CI   
5935  TEXTFT43            CI   
5936  TEXTFT44            CI   
5937  TEXTFT45            CI   

                                         schedule_label  \
0     Schedule RC-R Part I - Regulatory Capital Comp...   
1     Schedule RC-R Part I - Regulatory Capital Comp...   
2     Schedule RC-R Part I - Regulatory Capital Comp...   
3     Schedule RC-R Part I - Regulatory Capital Comp...   
4     Schedule RC-R Part I - Regulatory Capital Comp...   
...                                                 ...   
5933     Schedule RI-E - Explanations (Form Type - 051)   
5934               Contact Information(Form Type - 051)   
5935               Contact Information(Form Type - 051)   
5936               Contact Information(Form Type - 051)   
5937               Contact Information(Form Type - 051)   

                                             mdrm_label  form_type  
0     Total assets (Schedule RC, item 12); (must be ...  FFIEC 031  
1                       Allocated transfer risk reserve  FFIEC 031  
2     Total capitalTotal capital (sum of items 26 an...  FFIEC 031  
3     Allowance for loan and lease losses and eligib...  FFIEC 031  
4     Tier 2 capitalTier 2 capital (greater of item ...  FFIEC 031  
...                                                 ...        ...  
5933  Discontinued operations and applicable income ...  FFIEC 051  
5934  Chief Executive Officer Contact InformationChi...  FFIEC 051  
5935  Chief Executive Officer Contact InformationChi...  FFIEC 051  
5936  Chief Executive Officer Contact InformationChi...  FFIEC 051  
5937  Chief Executive Officer Contact InformationChi...  FFIEC 051  

[5938 rows x 5 columns]

In [12]:
df1 = extract_taxonomy_data(pd.read_json("C:\\Users\\whill\\call-report-database\\python\\cdr_taxonomy_xbrl_to_json\\example\\031_2001-03-31.json"),'FFIEC 031')
df2 = extract_taxonomy_data(pd.read_json("C:\\Users\\whill\\call-report-database\\python\\cdr_taxonomy_xbrl_to_json\\example\\041_2001-03-31.json"),'FFIEC 041')
older_df = pd.concat([df1,df2]).reset_index(drop=True)
older_df

mdrm_code schedule_type  \
0     RCFD0010           RCR   
1     RCFD0022           RCA   
2     RCFD0071            RC   
3     RCFD0073           RCA   
4     RCFD0074           RCA   
...        ...           ...   
1716  TEXT8902            CI   
1717  TEXT9116            CI   
1718  TEXTA545           RCO   
1719  TEXTB526           RIE   
1720  TEXTB527           RIE   

                                         schedule_label  \
0                    Schedule RC-R - Regulatory Capital   
1     Schedule RC-A - Cash and Balances Due From Dep...   
2                           Schedule RC - Balance Sheet   
3     Schedule RC-A - Cash and Balances Due From Dep...   
4     Schedule RC-A - Cash and Balances Due From Dep...   
...                                                 ...   
1716                                Contact Information   
1717                                Contact Information   
1718  Schedule RC-O - Other Data for Deposit Insuran...   
1719                       Schedule RI-E - Explanations   
1720                       Schedule RI-E - Explanations   

                                             mdrm_label  form_type  
0     Cash and balances due from depository institut...  FFIEC 031  
1     Cash items in process of collection, unposted ...  FFIEC 031  
2     Cash and balances due from depository institut...  FFIEC 031  
3     Balances due from banks in foreign countries a...  FFIEC 031  
4     Balances due from banks in foreign countries a...  FFIEC 031  
...                                                 ...        ...  
1716  Person to whom questions about the Reports of ...  FFIEC 041  
1717  Person to whom questions about the Reports of ...  FFIEC 041  
1718  Has the reporting institution been consolidate...  FFIEC 041  
1719  Restatements due to corrections of material ac...  FFIEC 041  
1720  Restatements due to corrections of material ac...  FFIEC 041  

[1721 rows x 5 columns]

In [13]:
df_2000_2023 = pd.concat([merged_df,older_df]).reset_index(drop=True)
df_2000_2023

mdrm_code schedule_type  \
0     RCFA2170          RCRI   
1     RCFA3128          RCRI   
2     RCFA3792          RCRI   
3     RCFA5310          RCRI   
4     RCFA5311          RCRI   
...        ...           ...   
7654  TEXT8902            CI   
7655  TEXT9116            CI   
7656  TEXTA545           RCO   
7657  TEXTB526           RIE   
7658  TEXTB527           RIE   

                                         schedule_label  \
0     Schedule RC-R Part I - Regulatory Capital Comp...   
1     Schedule RC-R Part I - Regulatory Capital Comp...   
2     Schedule RC-R Part I - Regulatory Capital Comp...   
3     Schedule RC-R Part I - Regulatory Capital Comp...   
4     Schedule RC-R Part I - Regulatory Capital Comp...   
...                                                 ...   
7654                                Contact Information   
7655                                Contact Information   
7656  Schedule RC-O - Other Data for Deposit Insuran...   
7657                       Schedule RI-E - Explanations   
7658                       Schedule RI-E - Explanations   

                                             mdrm_label  form_type  
0     Total assets (Schedule RC, item 12); (must be ...  FFIEC 031  
1                       Allocated transfer risk reserve  FFIEC 031  
2     Total capitalTotal capital (sum of items 26 an...  FFIEC 031  
3     Allowance for loan and lease losses and eligib...  FFIEC 031  
4     Tier 2 capitalTier 2 capital (greater of item ...  FFIEC 031  
...                                                 ...        ...  
7654  Person to whom questions about the Reports of ...  FFIEC 041  
7655  Person to whom questions about the Reports of ...  FFIEC 041  
7656  Has the reporting institution been consolidate...  FFIEC 041  
7657  Restatements due to corrections of material ac...  FFIEC 041  
7658  Restatements due to corrections of material ac...  FFIEC 041  

[7659 rows x 5 columns]

In [14]:
df_2000_2023.drop_duplicates(subset=['mdrm_code', 'form_type'],keep='first',inplace=True,ignore_index=True)

In [15]:
rows_to_drop = np.where(df_2000_2023['mdrm_code'].str.contains('TE', regex=True))
df_2000_2023.drop(index=rows_to_drop[0],inplace =True)

In [16]:
df_2000_2023

mdrm_code schedule_type  \
0     RCFA2170          RCRI   
1     RCFA3128          RCRI   
2     RCFA3792          RCRI   
3     RCFA5310          RCRI   
4     RCFA5311          RCRI   
...        ...           ...   
6621  RIADA517            RI   
6622  RIADA518            RI   
6623  RIADB490            RI   
6624  RIADB516          RIBI   
6625  RIADB517          RIBI   

                                         schedule_label  \
0     Schedule RC-R Part I - Regulatory Capital Comp...   
1     Schedule RC-R Part I - Regulatory Capital Comp...   
2     Schedule RC-R Part I - Regulatory Capital Comp...   
3     Schedule RC-R Part I - Regulatory Capital Comp...   
4     Schedule RC-R Part I - Regulatory Capital Comp...   
...                                                 ...   
6621                     Schedule RI - Income Statement   
6622                     Schedule RI - Income Statement   
6623                     Schedule RI - Income Statement   
6624  Schedule RI-B, Part I - Charge-offs and Recove...   
6625  Schedule RI-B, Part I - Charge-offs and Recove...   

                                             mdrm_label  form_type  
0     Total assets (Schedule RC, item 12); (must be ...  FFIEC 031  
1                       Allocated transfer risk reserve  FFIEC 031  
2     Total capitalTotal capital (sum of items 26 an...  FFIEC 031  
3     Allowance for loan and lease losses and eligib...  FFIEC 031  
4     Tier 2 capitalTier 2 capital (greater of item ...  FFIEC 031  
...                                                 ...        ...  
6621  Interest expense:Interest on deposits:Nontrans...  FFIEC 041  
6622  Interest expense:Interest on deposits:Nontrans...  FFIEC 041  
6623  Noninterest income:Investment banking, advisor...  FFIEC 041  
6624  Loans to individuals for household, family, an...  FFIEC 041  
6625  Loans to individuals for household, family, an...  FFIEC 041  

[6329 rows x 5 columns]

In [17]:
df_2000_2023['schedule_type'][df_2000_2023['mdrm_code']=='RCFA2170'].values[0]

'RCRI'

In [29]:
if 'RCFA2170' in df_2000_2023['mdrm_code'].values:
    print (True)

True


In [18]:
result_df = mdrm_filtered.join(df_2000_2023.set_index(['mdrm_code', 'form_type']), on=['mdrm_code','form_type'],how='left')
result_df[~result_df['schedule_type'].isnull()]

Mnemonic Item Code mdrm_code      Start Date         End Date  \
50643     RCFA      2170  RCFA2170  3/31/2020 0:00  12/31/9999 0:00   
50644     RCFA      3128  RCFA3128  3/31/2020 0:00  12/31/9999 0:00   
50645     RCFA      3792  RCFA3792  3/31/2014 0:00  12/31/9999 0:00   
50646     RCFA      5310  RCFA5310  3/31/2014 0:00  12/31/9999 0:00   
50647     RCFA      5311  RCFA5311  3/31/2014 0:00  12/31/9999 0:00   
...        ...       ...       ...             ...              ...   
69998     RSSD      9130  RSSD9130  3/31/2017 0:00  12/31/9999 0:00   
70063     RSSD      9200  RSSD9200  9/30/2005 0:00  12/31/9999 0:00   
70065     RSSD      9200  RSSD9200  3/31/2017 0:00  12/31/9999 0:00   
70084     RSSD      9220  RSSD9220  9/30/2005 0:00  12/31/9999 0:00   
70086     RSSD      9220  RSSD9220  3/31/2017 0:00  12/31/9999 0:00   

                                               Item Name Confidentiality  \
50643                                       TOTAL ASSETS               N   
50644                   ALLOCATED TRANSFER RISK RESERVES               N   
50645  TOTAL QUALIFYING CAPITAL ALLOWABLE UNDER THE R...               N   
50646        GENERAL LOAN AND LEASE VALUATION ALLOWANCES               N   
50647                     TIER 2 (SUPPLEMENTARY) CAPITAL               N   
...                                                  ...             ...   
69998                                CITY/TOWN TEXT NAME               N   
70063                             ABBREVIATED STATE NAME               N   
70065                             ABBREVIATED STATE NAME               N   
70084                  PHYSICAL ZIP/FOREIGN MAILING CODE               N   
70086                  PHYSICAL ZIP/FOREIGN MAILING CODE               N   

      ItemType  form_type                                        Description  \
50643        F  FFIEC 031  The sum of all asset items. This item must equ...   
50644        F  FFIEC 031   Section 905 of the International Lending Supe...   
50645        F  FFIEC 031  Includes the amount of the bank's total risk-b...   
50646        F  FFIEC 031   Includes general loan and lease valuation all...   
50647        F  FFIEC 031  Includes the sum of 'Qualifying Subordinated D...   
...        ...        ...                                                ...   
69998        S  FFIEC 051  The name of the city or town in which the enti...   
70063        S  FFIEC 031  Two character state abbreviation as assigned t...   
70065        S  FFIEC 051  Two character state abbreviation as assigned t...   
70084        S  FFIEC 031  The zip code or foreign mailing code of the ph...   
70086        S  FFIEC 051  The zip code or foreign mailing code of the ph...   

                                          SeriesGlossary schedule_type  \
50643   Regulatory Capital Components and Ratios for ...          RCRI   
50644   Regulatory Capital Components and Ratios for ...          RCRI   
50645   Regulatory Capital Components and Ratios for ...          RCRI   
50646   Regulatory Capital Components and Ratios for ...          RCRI   
50647   Regulatory Capital Components and Ratios for ...          RCRI   
...                                                  ...           ...   
69998                                                              ENT   
70063                                                              ENT   
70065                                                              ENT   
70084                                                              ENT   
70086                                                              ENT   

                                          schedule_label  \
50643  Schedule RC-R Part I - Regulatory Capital Comp...   
50644  Schedule RC-R Part I - Regulatory Capital Comp...   
50645  Schedule RC-R Part I - Regulatory Capital Comp...   
50646  Schedule RC-R Part I - Regulatory Capital Comp...   
50647  Schedule RC-R Part I - Regulatory Capital Comp...   
...                           

In [32]:
master_data_type_list = match_data_type(id_rssd_set,set(result_df['mdrm_code']))
datatype_df = pd.DataFrame(master_data_type_list,columns=['mdrm_code', 'data_type'])
datatype_df

The time of code execution begin is :  Wed Nov 29 15:50:42 2023
The time of code execution end was :  Wed Nov 29 18:07:37 2023


mdrm_code data_type
0     RCONA565       int
1     RIADJJ28       int
2     RIAD3217       int
3     RCON3049       int
4     RCONA575       int
...        ...       ...
2990  RCONL197       int
2991  RCFDL194       int
2992  RCFDL196       int
2993  RCFDK194       int
2994  RCFDK193       int

[2995 rows x 2 columns]

In [36]:
datatype_df[datatype_df['data_type']=='bool']

mdrm_code data_type
34    RCONFT07      bool
40    RCONFT03      bool
46    RCONA346      bool
48    RIADA530      bool
51    RCONFT06      bool
93    RCONFT11      bool
98    RCONB867      bool
195   RCONFT00      bool
209   RCONFT09      bool
223   RCONKX18      bool
281   RCONFT16      bool
304   RCONK656      bool
311   RCON6979      bool
341   RCONPG75      bool
407   RCONK659      bool
425   RIAD4769      bool
448   RCONA345      bool
507   RCONFT13      bool
646   RCONG463      bool
953   RCONB569      bool
1006  RCON6860      bool
1053  RCON6999      bool
1176  RCONP752      bool
1209  RCONG464      bool
1521  RCFDA345      bool
1606  RCFDB867      bool
1679  RCFDK659      bool
2391  RCFDB569      bool
2463  RCFDK656      bool
2544  RCFDA346      bool
2667  RCONL135      bool
2791  RCON4088      bool
2844  RCFD4088      bool

In [29]:
id_rssd_set.

TypeError: 'set' object is not subscriptable

In [26]:
master_data_type_list

NameError: name 'master_data_type_list' is not defined

In [37]:
final_df = result_df.join(datatype_df.set_index(['mdrm_code']), on=['mdrm_code'],how='left')
final_df

Mnemonic Item Code mdrm_code       Start Date         End Date  \
10710     CALL      8002  CALL8002   3/31/1984 0:00   3/31/2006 0:00   
10714     CALL      8786  CALL8786   3/31/1984 0:00  12/31/9999 0:00   
10716     CALL      8787  CALL8787  12/31/1959 0:00  12/31/9999 0:00   
10717     CALL      8798  CALL8798  12/31/1959 0:00  12/31/9999 0:00   
10718     CALL      8799  CALL8799  12/31/1959 0:00  12/31/2000 0:00   
...        ...       ...       ...              ...              ...   
69998     RSSD      9130  RSSD9130   3/31/2017 0:00  12/31/9999 0:00   
70063     RSSD      9200  RSSD9200   9/30/2005 0:00  12/31/9999 0:00   
70065     RSSD      9200  RSSD9200   3/31/2017 0:00  12/31/9999 0:00   
70084     RSSD      9220  RSSD9220   9/30/2005 0:00  12/31/9999 0:00   
70086     RSSD      9220  RSSD9220   3/31/2017 0:00  12/31/9999 0:00   

                               Item Name Confidentiality ItemType  form_type  \
10710          REPORTING STATUS (W-CODE)               N        F  FFIEC 031   
10714               REPORTING LEVEL CODE               N        S  FFIEC 031   
10716           REPORTING EXCEPTION CODE               N        F  FFIEC 031   
10717                     FDR TIME STAMP               Y        F  FFIEC 031   
10718                ARCHIVAL DATE STAMP               Y        F  FFIEC 031   
...                                  ...             ...      ...        ...   
69998                CITY/TOWN TEXT NAME               N        S  FFIEC 051   
70063             ABBREVIATED STATE NAME               N        S  FFIEC 031   
70065             ABBREVIATED STATE NAME               N        S  FFIEC 051   
70084  PHYSICAL ZIP/FOREIGN MAILING CODE               N        S  FFIEC 031   
70086  PHYSICAL ZIP/FOREIGN MAILING CODE               N        S  FFIEC 051   

                                             Description  \
10710  This item is an asset threshold used to determ...   
10714  Effective 10/24/2011; the FR 2572 information ...   
10716  Effective 10/24/2011; the FR 2572 information ...   
10717  Effective 10/24/2011; the FR 2572 information ...   
10718  The date (ccyymmdd) the data were last updated...   
...                                                  ...   
69998  The name of the city or town in which the enti...   
70063  Two character state abbreviation as assigned t...   
70065  Two character state abbreviation as assigned t...   
70084  The zip code or foreign mailing code of the ph...   
70086  The zip code or foreign mailing code of the ph...   

                                          SeriesGlossary schedule_type  \
10710  \r\nThe following items may be accessed by the...           NaN   
10714  \r\nThe following items may be accessed by the...           NaN   
10716  \r\nThe following items may be accessed by the...           NaN   
10717  \r\nThe following items may be accessed by the...           NaN   
10718  \r\nThe following items may be accessed by the...           NaN   
...                                                  ...           ...   
69998                                                              ENT   
70063                                                              ENT   
70065                                                              ENT   
70084                                                              ENT   
70086                                                              ENT   

                                      schedule_label          mdrm_label  \
10710                                            NaN                 NaN   
10714                                            NaN                 NaN   
10716                                            NaN                 NaN   
10717                                            NaN                 NaN   
10718                                            NaN                 NaN   
...                                              ...                 ...   
69998  Bank Demographic Information(Fo

In [42]:
final_df.to_csv("C:\\Users\\whill\\call-report-database\\MDRM_CSV_Filtered_Schedules_Data_Types.csv")

In [41]:
len(final_df[~final_df['data_type'].isnull()])

3525

In [18]:
match_df = result_df[(result_df['schedule_type'].isnull()) & (result_df['End Date'] == '12/31/9999 0:00')]
result_df[result_df['Item Code']=='G813']

Mnemonic Item Code mdrm_code      Start Date         End Date  \
54086     RCFD      G813  RCFDG813  6/30/2009 0:00  12/31/9999 0:00   

                                               Item Name Confidentiality  \
54086  ALL HOLDINGS OF MBS:RESIDENTIAL MORTGAGE PASS-...               N   

      ItemType  form_type                                        Description  \
54086        D  FFIEC 031  FFIEC031 RC-B -Securities Derived Formula: SUM...   

      SeriesGlossary schedule_type schedule_label mdrm_label  
54086            NaN           NaN            NaN        NaN

# just set things to 'Missing' Schedule Name, work on developing the code for creating the tables and setting up a new snowflake instance, then work on code for pulling the data merging it together by bank and then looping through the mdrm_code and tying the value to a specific table. 

# we also need to think about the types of the values, see if we can loop through a list of bank's first, find a data type and then create logic to develop the create table queries 

In [14]:
result_df.to_csv("C:\\Users\\whill\\call-report-database\\MDRM_CSV_Filtered_Schedules.csv")

In [13]:
schedule_list = set(result_df['schedule_type'][~result_df['schedule_type'].isnull()])

{'ENT',
 'LEO',
 'NARR',
 'RC',
 'RCA',
 'RCB',
 'RCCI',
 'RCCII',
 'RCD',
 'RCE',
 'RCEI',
 'RCEII',
 'RCF',
 'RCG',
 'RCH',
 'RCI',
 'RCK',
 'RCL',
 'RCM',
 'RCN',
 'RCO',
 'RCP',
 'RCQ',
 'RCR',
 'RCRI',
 'RCRII',
 'RCS',
 'RCT',
 'RCV',
 'RI',
 'RIA',
 'RIBI',
 'RIBII',
 'RIC',
 'RICI',
 'RICII',
 'RID',
 'RIE',
 'SU'}

In [38]:
result_df[result_df['schedule_type'].isin(['ENT'])]

Mnemonic Item Code mdrm_code      Start Date         End Date  \
62559     RCON      9224  RCON9224  3/31/2017 0:00  12/31/9999 0:00   
62578     RCON      9999  RCON9999  3/31/2017 0:00  12/31/9999 0:00   
69882     RSSD      9017  RSSD9017  9/30/2005 0:00  12/31/9999 0:00   
69884     RSSD      9017  RSSD9017  3/31/2017 0:00  12/31/9999 0:00   
69996     RSSD      9130  RSSD9130  9/30/2005 0:00  12/31/9999 0:00   
69998     RSSD      9130  RSSD9130  3/31/2017 0:00  12/31/9999 0:00   
70063     RSSD      9200  RSSD9200  9/30/2005 0:00  12/31/9999 0:00   
70065     RSSD      9200  RSSD9200  3/31/2017 0:00  12/31/9999 0:00   
70084     RSSD      9220  RSSD9220  9/30/2005 0:00  12/31/9999 0:00   
70086     RSSD      9220  RSSD9220  3/31/2017 0:00  12/31/9999 0:00   

                               Item Name Confidentiality ItemType  form_type  \
62559            LEGAL ENTITY IDENTIFIER               N        F  FFIEC 051   
62578       REPORTING DATE (CC;YR;MO;DA)               N        F  FFIEC 051   
69882                         LEGAL NAME               N        S  FFIEC 031   
69884                         LEGAL NAME               N        S  FFIEC 051   
69996                CITY/TOWN TEXT NAME               N        S  FFIEC 031   
69998                CITY/TOWN TEXT NAME               N        S  FFIEC 051   
70063             ABBREVIATED STATE NAME               N        S  FFIEC 031   
70065             ABBREVIATED STATE NAME               N        S  FFIEC 051   
70084  PHYSICAL ZIP/FOREIGN MAILING CODE               N        S  FFIEC 031   
70086  PHYSICAL ZIP/FOREIGN MAILING CODE               N        S  FFIEC 051   

                                             Description  \
62559  The Legal Entity Identifier (LEI) is a 20-digi...   
62578  Effective 10/24/2011; the FR 2572 information ...   
69882  Effective 10/24/2011; the FR 2572 information ...   
69884  Effective 10/24/2011; the FR 2572 information ...   
69996  The name of the city or town in which the enti...   
69998  The name of the city or town in which the enti...   
70063  Two character state abbreviation as assigned t...   
70065  Two character state abbreviation as assigned t...   
70084  The zip code or foreign mailing code of the ph...   
70086  The zip code or foreign mailing code of the ph...   

                                          SeriesGlossary schedule_type  \
62559   The mnemonic prefixes in this file represent ...           ENT   
62578   The mnemonic prefixes in this file represent ...           ENT   
69882                                                              ENT   
69884                                                              ENT   
69996                                                              ENT   
69998                                                              ENT   
70063                                                              ENT   
70065                                                              ENT   
70084                                                              ENT   
70086                                                              ENT   

                                      schedule_label  \
62559  Bank Demographic Information(Form Type - 051)   
62578  Bank Demographic Information(Form Type - 051)   
69882  Bank Demographic Information(Form Type - 031)   
69884  Bank Demographic Information(Form Type - 051)   
69996  Bank Demographic Information(Form Type - 031)   
69998  Bank Demographic Information(Form Type - 051)   
70063  Bank Demographic Information(Form Type - 031)   
70065  Bank Demographic Information(Form Type - 051)   
70084  Bank Demographic Information(Form Type - 031)   
70086  Bank Demographic Information(Form Type - 051)   

                                              mdrm_label  
62559  Legal Entity Identifier (LEI) (Report only if ...  
62578                                     Reporting date  
69882                                Legal title of bank  
69884         

In [49]:
start_list = ['IDRSSD', 'RCON9999']
result_df['mdrm_code'][result_df['schedule_type'].isin(['ENT'])].to_list()
start_list.extend(result_df['mdrm_code'][result_df['schedule_type'].isin(['ENT'])].to_list())
columns_set = set(start_list)
columns_set

{'IDRSSD',
 'RCON9224',
 'RCON9999',
 'RSSD9017',
 'RSSD9130',
 'RSSD9200',
 'RSSD9220'}

In [52]:
data_list = ['VARCHAR', 'DATE']
for i in range(len(columns_set)-2):
    data_list.append('NUMBER')
data_list

['VARCHAR', 'DATE', 'NUMBER', 'NUMBER', 'NUMBER', 'NUMBER', 'NUMBER']

In [53]:
def create_table_query(table_name, columns, data_types):
    # Check if the lengths of the lists are the same
    if len(columns) != len(data_types):
        raise ValueError("Number of columns and data types must be the same.")

    # Construct the Create table query
    columns_str = ', '.join([f'{col} {data_type}' for col, data_type in zip(columns, data_types)])
    query = f'CREATE TABLE {table_name} ({columns_str});'

    return query

# Example usage:
table_name = 'example_table'
column_names = ['id', 'name', 'age']
data_types = ['INT', 'VARCHAR(255)', 'INT']

sql_query = create_table_query(table_name, columns_set, data_list)
print(sql_query)

CREATE TABLE example_table (IDRSSD VARCHAR, RCON9224 DATE, RSSD9200 NUMBER, RSSD9130 NUMBER, RCON9999 NUMBER, RSSD9220 NUMBER, RSSD9017 NUMBER);


In [ ]:
import pandas
from snowflake.connector.pandas_tools import write_pandas

# Create the connection to the Snowflake database.
cnx = snowflake.connector.connect(...)

# Create a DataFrame containing data about customers
df = pandas.DataFrame([('Mark', 10), ('Luke', 20)], columns=['name', 'balance'])

# Write the data from the DataFrame to the table named "customers".
success, nchunks, nrows, _ = write_pandas(cnx, df, 'customers')

In [ ]:
import snowflake.snowpark as snowpark
from snowflake.snowpark.functions import col

def main(session: snowpark.Session): 
    # CREATE TABLE
    create_table_query  = f"""
       CREATE OR REPLACE TABLE TEST_DB.TEST_SCHEMA.CALL_CENTER_COPY AS
       SELECT 
              CC_CALL_CENTER_ID
             ,CC_REC_START_DATE 
             ,CC_REC_END_DATE
       FROM TEST_DB.TEST_SCHEMA.CALL_CENTER;
    """
    session.sql(create_table_query).collect()
    # SELECT TABLE
    create_table_query  = f"""
       SELECT 
              *
       FROM TEST_DB.TEST_SCHEMA.CALL_CENTER_COPY;
    """
    call_center_table = session.sql(create_table_query).collect()
    return call_center_table

In [32]:
df_2000_2023['mdrm_code'].str.contains('TEXT').sum()

251

In [23]:
older_df.loc[older_df['mdrm_code'] == 'RIADB527']

mdrm_code schedule_type                schedule_label  \
852   RIADB527           RIE  Schedule RI-E - Explanations   
1675  RIADB527           RIE  Schedule RI-E - Explanations   

                                             mdrm_label  form_type  
852   Restatements due to corrections of material ac...  FFIEC 031  
1675  Restatements due to corrections of material ac...  FFIEC 041

In [25]:
older_df['mdrm_label'].iloc[1675]

'Restatements due to corrections of material accounting errors and changes in accounting principles (from Schedule RI-A)Itemized amount of Restatements'

In [8]:
merged_df.to_csv("C:\\Users\\whill\\call-report-database\\python\\cdr_taxonomy_xbrl_to_json\\example\\Taxonomies_example.csv")

In [4]:
final_df

mdrm_code schedule_type  \
0     RCFA2170          RCRI   
1     RCFA3128          RCRI   
2     RCFA3792          RCRI   
3     RCFA5310          RCRI   
4     RCFA5311          RCRI   
...        ...           ...   
2367  TEXTG561           RCQ   
2368  TEXTG571           RCQ   
2369  TEXTG576           RCQ   
2370  TEXTG581           RCQ   
2371  TEXTG586           RCQ   

                                         schedule_label  \
0     Schedule RC-R Part I - Regulatory Capital Comp...   
1     Schedule RC-R Part I - Regulatory Capital Comp...   
2     Schedule RC-R Part I - Regulatory Capital Comp...   
3     Schedule RC-R Part I - Regulatory Capital Comp...   
4     Schedule RC-R Part I - Regulatory Capital Comp...   
...                                                 ...   
2367  Schedule RC-Q - Assets and Liabilities Measure...   
2368  Schedule RC-Q - Assets and Liabilities Measure...   
2369  Schedule RC-Q - Assets and Liabilities Measure...   
2370  Schedule RC-Q - Assets and Liabilities Measure...   
2371  Schedule RC-Q - Assets and Liabilities Measure...   

                                             mdrm_label  
0     Total assets (Schedule RC, item 12); (must be ...  
1                       Allocated transfer risk reserve  
2     Total capitalTotal capital (sum of items 26 an...  
3     Allowance for loan and lease losses and eligib...  
4     Tier 2 capitalTier 2 capital (greater of item ...  
...                                                 ...  
2367                                 Describe component  
2368                                 Describe component  
2369                                 Describe component  
2370                                 Describe component  
2371                                 Describe component  

[2372 rows x 4 columns]

In [ ]:
df3=pd.read_json("C:\\Users\\whill\\call-report-database\\python\\cdr_taxonomy_xbrl_to_json\\example\\031_2023-09-30.json")
df3

In [5]:
sys.path.append(os.path.abspath("C:\\Users\\whill\\OneDrive\\Desktop\\Fall 2022\\ISYE 6740\\call-report-database\\python\\cdr_taxonomy_xbrl_to_json"))

In [15]:
import cdr_taxonomy_processor as cdr_processor

In [27]:
cdr_processor.process_zip("C:\\Users\\whill\\OneDrive\\Desktop\\Fall 2022\\ISYE 6740\\call-report-database\\python\\cdr_taxonomy_xbrl_to_json\\FFIEC CDR UBPR Bulk XBRL 03312022.zip")


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Loading ZIP file:  C:\Users\whill\OneDrive\Desktop\Fall 2022\ISYE 6740\call-report-database\python\cdr_taxonomy_xbrl_to_json\FFIEC CDR UBPR Bulk XBRL 03312022.zip
The included ZIP file does not appear to be a valid CDR taxonomy file.

Exiting with error
Traceback (most recent call last):
  File "C:\Users\whill\OneDrive\Desktop\Fall 2022\ISYE 6740\call-report-database\python\cdr_taxonomy_xbrl_to_json\cdr_taxonomy_processor.py", line 96, in process_zip
    cap_file = list(filter(lambda x: '-cap' in x, files_parsed))[0]
IndexError: list index out of range

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\whill\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-db84e2cba177>", line 1, in <module>
    cdr_processor.process_zip("C:\\Users\\whill\\OneDrive\\Desktop\\Fall 2022\\ISYE 6740\\call-report-database\

TypeError: object of type 'NoneType' has no len()

In [10]:
!python C:\\Users\\whill\\OneDrive\\Desktop\\Fall 2022\\ISYE 6740\\call-report-database\\python\\cdr_taxonomy_xbrl_to_json\\cdr_taxonomy_processor.py

python: can't open file 'C:\\Users\\whill\\OneDrive\\Desktop\\Fall': [Errno 2] No such file or directory


In [29]:
df= pd.json_normalize(df2["data"])
df

RCRI.line_ids.schedule.code   
0             ffiec031_pres-RCRI  \
1             ffiec031_pres-RCRI   
2             ffiec031_pres-RCRI   
3             ffiec031_pres-RCRI   
4             ffiec031_pres-RCRI   
...                          ...   
2367                         NaN   
2368                         NaN   
2369                         NaN   
2370                         NaN   
2371                         NaN   

                           RCRI.line_ids.schedule.label   
0     Schedule RC-R Part I - Regulatory Capital Comp...  \
1     Schedule RC-R Part I - Regulatory Capital Comp...   
2     Schedule RC-R Part I - Regulatory Capital Comp...   
3     Schedule RC-R Part I - Regulatory Capital Comp...   
4     Schedule RC-R Part I - Regulatory Capital Comp...   
...                                                 ...   
2367                                                NaN   
2368                                                NaN   
2369                                                NaN   
2370                                                NaN   
2371                                                NaN   

     RCRI.line_ids.extra_col_0.code   
0        ffiec031_pres-line-1010119  \
1        ffiec031_pres-line-1010106   
2        ffiec031_pres-line-1010113   
3        ffiec031_pres-line-1010112   
4        ffiec031_pres-line-1010118   
...                             ...   
2367                            NaN   
2368                            NaN   
2369                            NaN   
2370                            NaN   
2371                            NaN   

                        RCRI.line_ids.extra_col_0.label   
0     Total assets (Schedule RC, item 12); (must be ...  \
1                       Allocated transfer risk reserve   
2                                         Total capital   
3     Allowance for loan and lease losses and eligib...   
4                                        Tier 2 capital   
...                                                 ...   
2367                                                NaN   
2368                                                NaN   
2369                                                NaN   
2370                                                NaN   
2371                                                NaN   

     RCRI.column_ids.schedule.code   
0               ffiec031_pres-RCRI  \
1               ffiec031_pres-RCRI   
2               ffiec031_pres-RCRI   
3               ffiec031_pres-RCRI   
4               ffiec031_pres-RCRI   
...                            ...   
2367                           NaN   
2368                           NaN   
2369                           NaN   
2370                           NaN   
2371                           NaN   

                         RCRI.column_ids.schedule.label   
0     Schedule RC-R Part I - Regulatory Capital Comp...  \
1     Schedule RC-R Part I - Regulatory Capital Comp...   
2     Schedule RC-R Part I - Regulatory Capital Comp...   
3     Schedule RC-R Part I - Regulatory Capital Comp...   
4     Schedule RC-R Part I - Regulatory Capital Comp...   
...                                                 ...   
2367                                                NaN   
2368                                                NaN   
2369                                                NaN   
2370                                                NaN   
2371                                                NaN   

       RCRI.column_ids.colset.code  RCRI.column_ids.colset.label   
0     ffiec031_pres-RCRI-76-colset                           NaN  \
1     ffiec031_pres-RCRI-77-colset                           NaN   
2     ffiec031_pres-RCRI-77-colset                           NaN   
3     ffiec031_pres-RCRI-77-colset                           NaN   
4     ffiec031_pres-RCRI-77-colset                           NaN   
...                            ...                           ...   
2367                           NaN         

In [44]:
keys_list = list(df2["data"][0].keys())
subset_of_data = df2["data"][0][keys_list[0]]
#subset_of_data['line_ids']['schedule']['label']
subset_of_data['line_ids']['extra_col_0']['label']

2

In [48]:
re.split('_',df2.index[0])[1]

'RCFA2170'

In [23]:
data = df2['data']

# Create an empty DataFrame to store the results
result_df = pd.DataFrame()

# Iterate over keys in 'data'
for key, value in data.items():
    # Flatten the nested structures under the current key
    df_temp = pd.json_normalize(value, sep='_')
    
    # Check if DataFrame has columns before setting the index
    if not df_temp.empty:
        df_temp.set_index(key, inplace=True)
    
        # Add 'schedule' field as metadata from df2
        df_temp['schedule'] = df2.get('schedule', None)
    
        # Add the DataFrame to the result DataFrame
        result_df = pd.concat([result_df, df_temp])


# Set 'schedule' as the first column (optional)
result_df = result_df[['schedule'] + [col for col in result_df.columns if col != 'schedule']]

print(result_df)

KeyError: "None of ['cc_RCFA2170'] are in the columns"

cc_RCFA2170    {'RCRI': {'line_ids': {'schedule': {'code': 'f...
cc_RCFA3128    {'RCRI': {'line_ids': {'schedule': {'code': 'f...
cc_RCFA3792    {'RCRI': {'line_ids': {'schedule': {'code': 'f...
cc_RCFA5310    {'RCRI': {'line_ids': {'schedule': {'code': 'f...
cc_RCFA5311    {'RCRI': {'line_ids': {'schedule': {'code': 'f...
                                     ...                        
cc_TEXTG561    {'RCQ': {'line_ids': {'schedule': {'code': 'ff...
cc_TEXTG571    {'RCQ': {'line_ids': {'schedule': {'code': 'ff...
cc_TEXTG576    {'RCQ': {'line_ids': {'schedule': {'code': 'ff...
cc_TEXTG581    {'RCQ': {'line_ids': {'schedule': {'code': 'ff...
cc_TEXTG586    {'RCQ': {'line_ids': {'schedule': {'code': 'ff...
Name: data, Length: 2372, dtype: object

In [31]:
df['data'][0]

{'RCRI': {'line_ids': {'schedule': {'code': 'ffiec031_pres-RCRI',
    'label': 'Schedule RC-R Part I - Regulatory Capital Components and Ratios(Form Type - 031)'},
   'extra_col_0': {'code': 'ffiec031_pres-line-1010119',
    'label': 'Total assets (Schedule RC, item 12); (must be less than $10 billion)'}},
  'column_ids': {'schedule': {'code': 'ffiec031_pres-RCRI',
    'label': 'Schedule RC-R Part I - Regulatory Capital Components and Ratios(Form Type - 031)'},
   'colset': {'code': 'ffiec031_pres-RCRI-76-colset', 'label': None},
   'column': {'code': 'ffiec031_pres-RCRI-76-column-A',
    'label': '(Column A) Amount'}},
  'reference': {'line': '32', 'column': 'A'}}}

In [149]:
new_dict ={}
for i in range(len(df)):
    clone= df['data'][i]
    line_ids_data = clone[next(iter(df['data'][i]))][keys[0]]
    if len(line_ids_data) > 2:
        print(line_ids_data)
        schedule_data = line_ids_data['schedule']
        schedule_label = schedule_data.get('label', None)
        extra_col_0_data = []
        for i in range(1,len(line_ids_data)):
            key = f'extra_col_{i}'
            extra_col_data = line_ids_data.get(key, {})
            extra_col_0_data.append(str(extra_col_data.get('label', None)))
        extra_col_0_label = ' '.join(filter(None, extra_col_0_data))
    else:
        schedule_data = line_ids_data['schedule']
        extra_col_0_data = line_ids_data['extra_col_0']
        schedule_label = schedule_data.get('label', None)
        extra_col_0_label = extra_col_0_data.get('label', None)
    new_dict.update({re.split('_',df.index[i])[1]:{'Schedule':next(iter(df['data'][i])),
                                                   'MDRM_Label':f'{extra_col_0_label}',
                                                   'Schedule_label':f'{schedule_label}'}})
    
#new_dict

{'schedule': {'code': 'ffiec031_pres-RCRI', 'label': 'Schedule RC-R Part I - Regulatory Capital Components and Ratios(Form Type - 031)'}, 'extra_col_0': {'code': 'ffiec031_pres-line-1010113', 'label': 'Total capital'}, 'extra_col_1': {'code': 'ffiec031_pres-line-1010098', 'label': 'Total capital (sum of items 26 and 46.a)'}}
{'schedule': {'code': 'ffiec031_pres-RCRI', 'label': 'Schedule RC-R Part I - Regulatory Capital Components and Ratios(Form Type - 031)'}, 'extra_col_0': {'code': 'ffiec031_pres-line-1010112', 'label': 'Allowance for loan and lease losses and eligible credit reserves includable in tier 2 capital'}, 'extra_col_1': {'code': 'ffiec031_pres-line-1010093', 'label': 'Allowance for loan and lease losses includable in tier 2 capital'}}
{'schedule': {'code': 'ffiec031_pres-RCRI', 'label': 'Schedule RC-R Part I - Regulatory Capital Components and Ratios(Form Type - 031)'}, 'extra_col_0': {'code': 'ffiec031_pres-line-1010118', 'label': 'Tier 2 capital'}, 'extra_col_1': {'code'

In [139]:
[np.where(df.index=='cc_RCFDD965')]

[(array([498], dtype=int64),)]

In [150]:
new_dict

{'RCFA2170': {'Schedule': 'RCRI',
  'MDRM_Label': 'Total assets (Schedule RC, item 12); (must be less than $10 billion)',
  'Schedule_label': 'Schedule RC-R Part I - Regulatory Capital Components and Ratios(Form Type - 031)'},
 'RCFA3128': {'Schedule': 'RCRI',
  'MDRM_Label': 'Allocated transfer risk reserve',
  'Schedule_label': 'Schedule RC-R Part I - Regulatory Capital Components and Ratios(Form Type - 031)'},
 'RCFA3792': {'Schedule': 'RCRI',
  'MDRM_Label': 'FAX None',
  'Schedule_label': 'Contact Information(Form Type - 031)'},
 'RCFA7204': {'Schedule': 'RCRI',
  'MDRM_Label': 'Interest on deposits: Interest on deposits in domestic offices: Nontransaction accounts: Time deposits of more than $250,000 None',
  'Schedule_label': 'Schedule RI - Income Statement(Form Type - 031)'},
 'RCFA7205': {'Schedule': 'RCRI',
  'MDRM_Label': 'Total capital ratio (Column A: item 47.a divided by item 48.a) (Advanced approaches institutions that exit parallel run only: Column B: item 47.b divided 

In [145]:
df['data'][498]

{'RCRII': {'line_ids': {'schedule': {'code': 'ffiec031_pres-RCRII',
    'label': 'Schedule RC-R Part II - Risk-Weighted Assets(Form Type - 031)'},
   'extra_col_0': {'code': 'ffiec031_pres-line-1010130',
    'label': 'Securities:'},
   'extra_col_1': {'code': 'ffiec031_pres-line-1010129',
    'label': 'Held-to-maturity securities'}},
  'column_ids': {'schedule': {'code': 'ffiec031_pres-RCRII',
    'label': 'Schedule RC-R Part II - Risk-Weighted Assets(Form Type - 031)'},
   'colset': {'code': 'ffiec031_pres-RCRII-80-colset', 'label': None},
   'column': {'code': 'ffiec031_pres-RCRII-80-column-I',
    'label': '(Column I) Allocation by Risk-Weight Category 100%'}},
  'reference': {'line': '2a', 'column': 'I'}}}

In [37]:
re.split('_',df.index[0])[1]

'RCFA2170'

In [79]:
next(iter(df['data'][0]))

'RCRI'

In [123]:
line_ids_data = test[next(iter(df['data'][0]))][keys[0]]
schedule_data = line_ids_data['schedule']
extra_col_0_data = line_ids_data['extra_col_0']
schedule_label = schedule_data.get('label', None)
extra_col_0_label = extra_col_0_data.get('label', None)
print("Schedule Label:", schedule_label)
print("Extra_col_0 Label:", extra_col_0_label)

Schedule Label: Schedule RC-R Part I - Regulatory Capital Components and Ratios(Form Type - 031)
Extra_col_0 Label: Total assets (Schedule RC, item 12); (must be less than $10 billion)


In [127]:
line_ids_data

{'schedule': {'code': 'ffiec031_pres-RCQ',
  'label': 'Schedule RC-Q - Assets and Liabilities Measured at Fair Value on a Recurring Basis(Form Type - 031)'},
 'extra_col_0': {'code': 'ffiec031_pres-line-1010016',
  'label': 'Describe component'}}

In [99]:
for key, value in test[next(iter(df['data'][0]))][keys[0]]:
    print(key, '->', value)

ValueError: too many values to unpack (expected 2)

In [94]:
test = df['data'][0]
for i in test:
    keys= list(test[i].keys())

In [95]:
keys

['line_ids', 'column_ids', 'reference']

In [ ]:
    if test[i].keys() == 'line_ids':
        print(test[i].value())

In [117]:
for ids, info in test.items():
    print("\ntop :", ids)
    for key in info:
        if key == 'line_ids':
            print(list(info[key].values()))



top : RCRI
[{'code': 'ffiec031_pres-RCRI', 'label': 'Schedule RC-R Part I - Regulatory Capital Components and Ratios(Form Type - 031)'}, {'code': 'ffiec031_pres-line-1010119', 'label': 'Total assets (Schedule RC, item 12); (must be less than $10 billion)'}]
